In [3]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from subprocess import check_output
from pyspark import SparkFiles
import collections
import random

spark_jars = "org.apache.hadoop:hadoop-azure:3.3.1,com.microsoft.azure:azure-storage:8.6.6"

spark = (SparkSession.builder
    .master("spark://ec1266a27642:7077")
    .appName("test-edu")
    .config("spark.jars.packages", spark_jars)\
    .config("spark.executor.memory", "4g")\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
    .getOrCreate()
        )


In [18]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
df_voos = spark.read.csv('abfss://anac@ufmstrtadls.dfs.core.windows.net/RAW/2022', sep=';', header=True)
filtro_destino = (df_voos.sg_icao_destino == 'SBCG')
dfvoos_2022 = df_voos.filter(filtro_destino).select(['nm_empresa', 'sg_empresa_iata', 'nr_mes_partida_real', 'nr_voo', 'sg_icao_origem', 'sg_icao_destino'])\
                .groupby(['nm_empresa', 'sg_empresa_iata',  'nr_voo', 'sg_icao_origem', 'sg_icao_destino', 'nr_mes_partida_real'])\
                .count().sort('count', ascending=False)

In [20]:
dfvoos_2022.write.parquet('abfss://anac@ufmstrtadls.dfs.core.windows.net/TRUSTED/ANAC/VOOS_REALIZADOS_2022', mode='overwrite')